In [89]:
import keyring
from pprint import pprint

api_key_irp = keyring.get_password("irp_api_key", "birdblues")
api_secret_irp = keyring.get_password("irp_api_secret", "birdblues")

api_key_isa = keyring.get_password("isa_api_key", "birdblues")
api_secret_isa = keyring.get_password("isa_api_secret", "birdblues")

api_key_0 = keyring.get_password("api_key_0", "birdblues")
api_secret_0 = keyring.get_password("api_secret_0","birdblues")

api_key_1 = keyring.get_password("api_key_1", "birdblues")
api_secret_1 = keyring.get_password("api_secret_1","birdblues")

api_key_2 = keyring.get_password("api_key_2", "birdblues")
api_secret_2 = keyring.get_password("api_secret_2","birdblues")

# print(f"api_key_irp = {api_key_irp}")
# print(f"api_secret_irp = {api_secret_irp}")
# print(f"api_key_isa = {api_key_isa}")
# print(f"api_secret_isa = {api_secret_isa}")
# print(f"api_key_0 = {api_key_0}")
# print(f"api_secret_0 = {api_secret_0}")
# print(f"api_key_1 = {api_key_1}")
# print(f"api_secret_1 = {api_secret_1}")
# print(f"api_key_2 = {api_key_2}")
# print(f"api_secret_2 = {api_secret_2}")

In [90]:
acc_no_0 = "63981494-01" # 위탁
acc_no_1 = "64267316-22" # 연금저축 1
acc_no_2 = "64012548-22" # 연금저축 2
acc_no_isa = "64043278-01" # ISA
acc_no_irp = "64012548-29" # IRP

In [91]:
import keyring

token_0 = keyring.get_password("access_token_0", "birdblues")
token_1 = keyring.get_password("access_token_1", "birdblues")
token_2 = keyring.get_password("access_token_2", "birdblues")
token_3 = keyring.get_password("access_token_3", "birdblues")
token_4 = keyring.get_password("access_token_4", "birdblues")

# print(f"token_0 = {token_0}")
# print(f"token_1 = {token_1}")
# print(f"token_2 = {token_2}")
# print(f"token_3 = {token_3}")
# print(f"token_4 = {token_4}")

In [92]:
import requests
import json

def get_balance(access_token, app_key, app_secret, acc_no):
    
    url_base = "https://openapi.koreainvestment.com:9443"
    path = "/uapi/domestic-stock/v1/trading/inquire-balance"
    url = f"{url_base}/{path}"

    headers = {
        "Content-Type": "application/json; charset=utf-8",
        "authorization": f"Bearer {access_token}",
        "appKey": app_key,
        "appSecret": app_secret,
        "tr_id": "TTTC8434R"
    }

    params = {
        "CANO": acc_no[:8],  # 계좌번호 앞 8지리
        "ACNT_PRDT_CD": acc_no[-2:],  # 계좌번호 뒤 2자리
        "AFHR_FLPR_YN": "N",  # 시간외단일가여부
        "OFL_YN": "",  # 공란
        "INQR_DVSN": "01",  # 조회구분
        "UNPR_DVSN": "01",  # 단가구분
        "FUND_STTL_ICLD_YN": "N",  # 펀드결제분포함여부
        "FNCG_AMT_AUTO_RDPT_YN": "N",  # 융자금액자동상환여부        
        "PRCS_DVSN": "00",  # 처리구분(00: 전일매매포함)
        "CTX_AREA_FK100": "",  # 연속조회검색조건
        "CTX_AREA_NK100": ""  # 연속조회키
    }
    res = requests.get(url, headers=headers, params=params)
    return res.json()
    

In [93]:
def get_balance_irp(access_token, app_key, app_secret, acc_no):
    
    url_base = "https://openapi.koreainvestment.com:9443"
    path = "/uapi/domestic-stock/v1/trading/pension/inquire-balance"
    url = f"{url_base}/{path}"

    headers = {
        "Content-Type": "application/json; charset=utf-8",
        "authorization": f"Bearer {access_token}",
        "appKey": app_key,
        "appSecret": app_secret,
        "tr_id": "TTTC2208R"
    }

    params = {
        "CANO": acc_no[:8],  # 계좌번호 앞 8지리
        "ACNT_PRDT_CD": acc_no[-2:],  # 계좌번호 뒤 2자리
        "ACCA_DVSN_CD":"00",
        "INQR_DVSN":"00",
        "CTX_AREA_FK100": "",  # 연속조회검색조건
        "CTX_AREA_NK100": ""  # 연속조회키
    }
    res = requests.get(url, headers=headers, params=params)
    return res.json()

In [94]:
json_0 = get_balance(token_0, api_key_0, api_secret_0, acc_no_0)
json_1 = get_balance(token_1, api_key_1, api_secret_1, acc_no_1)
json_2 = get_balance(token_2, api_key_2, api_secret_2, acc_no_2)
json_3 = get_balance(token_3, api_key_isa, api_secret_isa, acc_no_isa)
json_4 = get_balance_irp(token_4, api_key_irp, api_secret_irp, acc_no_irp)
json_4

{'ctx_area_fk100': '64012548^29^00^00^                                                                                  ',
 'ctx_area_nk100': '                                                                                                    ',
 'output1': [{'cblc_dvsn_name': '사용자',
   'prdt_name': 'KODEX 차이나CSI300',
   'pdno': '283580',
   'item_dvsn_name': '현금',
   'thdt_buyqty': '0',
   'thdt_sll_qty': '0',
   'hldg_qty': '20',
   'ord_psbl_qty': '20',
   'pchs_avg_pric': '10117.5000',
   'pchs_amt': '202350',
   'prpr': '10430',
   'evlu_amt': '208600',
   'evlu_pfls_amt': '6250',
   'evlu_erng_rt': '3.08870768'},
  {'cblc_dvsn_name': '사용자',
   'prdt_name': 'KODEX 200미국채혼합',
   'pdno': '284430',
   'item_dvsn_name': '현금',
   'thdt_buyqty': '0',
   'thdt_sll_qty': '0',
   'hldg_qty': '40',
   'ord_psbl_qty': '40',
   'pchs_avg_pric': '12608.7750',
   'pchs_amt': '504351',
   'prpr': '12955',
   'evlu_amt': '518200',
   'evlu_pfls_amt': '13849',
   'evlu_erng_rt': '2.74590513'},
  {

In [95]:
import pandas as pd

pd.options.display.float_format = '{:.2f}'.format

def get_balance_json(json):
    cash = json['output2'][0]['prvs_rcdl_excc_amt']
    ap = pd.DataFrame.from_records(json['output1'])
    if(ap.size > 0):
        ap = ap[["pdno", "prdt_name", "evlu_amt", "prpr", "hldg_qty", "pchs_amt", "pchs_avg_pric", "evlu_pfls_rt"]]
        ap.columns = ["코드", "이름", "평가금액", "단가", "수량", "매입금액", "매입단가", "수익률"]
        new_row = {"코드": "0", "이름": "현금", "평가금액": cash, "단가": 1, "수량": cash, "매입금액": cash, "매입단가": 1, "수익률": 0}
        new_row_df = pd.DataFrame([new_row])

        # concat을 사용하여 새로운 행 추가
        ap = pd.concat([ap, new_row_df], ignore_index=True)
        
        # 데이터 타입 변환
        ap["평가금액"] = pd.to_numeric(ap["평가금액"], errors='coerce')
        ap["매입금액"] = pd.to_numeric(ap["매입금액"], errors='coerce')
        ap["단가"] = pd.to_numeric(ap["단가"], errors='coerce')
        ap["매입단가"] = pd.to_numeric(ap["매입단가"], errors='coerce')
        ap["수량"] = pd.to_numeric(ap["수량"], errors='coerce')
        ap["수익률"] = pd.to_numeric(ap["수익률"], errors='coerce')
        ap["계좌"] = json['ctx_area_fk100'][:8] + '-' + json['ctx_area_fk100'][9:11]
    return ap

In [96]:
import pandas as pd


def get_balance_json_irp(json):
    cash = json['output2']['prvs_rcdl_excc_amt']
    ap = pd.DataFrame.from_records(json['output1'])
    if(ap.size > 0):
        ap = ap[["pdno", "prdt_name", "evlu_amt", "prpr", "hldg_qty", "pchs_amt", "pchs_avg_pric", "evlu_erng_rt"]]
        ap.columns = ["코드", "이름", "평가금액", "단가", "수량", "매입금액", "매입단가", "수익률"]
        new_row = {"코드": "0", "이름": "현금", "평가금액": cash, "단가": 1, "수량": cash, "매입금액": cash, "매입단가": 1, "수익률": 0}
        new_row_df = pd.DataFrame([new_row])

        # concat을 사용하여 새로운 행 추가
        ap = pd.concat([ap, new_row_df], ignore_index=True)

        # 데이터 타입 변환
        ap["평가금액"] = pd.to_numeric(ap["평가금액"], errors='coerce')
        ap["매입금액"] = pd.to_numeric(ap["매입금액"], errors='coerce')
        ap["단가"] = pd.to_numeric(ap["단가"], errors='coerce')
        ap["매입단가"] = pd.to_numeric(ap["매입단가"], errors='coerce')
        ap["수량"] = pd.to_numeric(ap["수량"], errors='coerce')
        ap["수익률"] = pd.to_numeric(ap["수익률"], errors='coerce')
        ap["계좌"] = json['ctx_area_fk100'][:8] + '-' + json['ctx_area_fk100'][9:11]
    return ap

In [97]:
df_0 = get_balance_json(json_0)
df_1 = get_balance_json(json_1)
df_2 = get_balance_json(json_2)
df_3 = get_balance_json(json_3)
df_4 = get_balance_json_irp(json_4)
df = pd.concat([df_0, df_1, df_2, df_3, df_4], ignore_index=True)
# df_0 = df_0.sort_values(by='이름')
df_1 = df_1.sort_values(by='이름')
df_2 = df_2.sort_values(by='이름')
df_3 = df_3.sort_values(by='이름')
df_4 = df_4.sort_values(by='이름')

total_1 = df_1["평가금액"].sum()
total_2 = df_2["평가금액"].sum()
total_3 = df_3["평가금액"].sum()
total_4 = df_4["평가금액"].sum()

In [98]:
df_1["비중"] = df_1["평가금액"] / total_1 * 100
df_1

,코드,이름,평가금액,단가,수량,매입금액,매입단가,수익률,계좌,비중
2,411060,ACE KRX금현물,6990750,15535,450,6669451,14821.00,4.81,64267316-22,14.44
9,476750,ACE 미국30년국채엔화노출액티브(H),1033000,10330,100,901663,9016.63,14.56,64267316-22,2.13
0,360200,ACE 미국S&P500,5469150,18050,303,5330680,17593.00,2.59,64267316-22,11.30
5,456880,ACE 미국달러SOFR금리(합성),3005100,11130,270,2945731,10910.11,2.01,64267316-22,6.21
10,480020,ACE 미국빅테크7+ 15%프리미엄분배(합성),2558920,10360,247,2901566,11747.23,-11.80,64267316-22,5.29
7,468380,KODEX iShares미국하이일드액티브,1635000,10900,150,1635050,10900.33,0.00,64267316-22,3.38
6,464540,RISE 25-03 회사채(AA-이상)액티브,2397235,51005,47,2394495,50946.70,0.11,64267316-22,4.95
8,473330,SOL 미국30년국채커버드콜(합성),10250240,10240,1001,9694000,9684.32,5.73,64267316-22,21.18
3,446720,SOL 미국배당다우존스,3190500,10635,300,3136694,10455.65,1.71,64267316-22,6.59
4,452360,SOL 미국배당다우존스(H),2164000,10820,200,2138084,10690.42,1.21,64267316-22,4.47


In [99]:
total_1

48403397

In [100]:
df_2

,코드,이름,평가금액,단가,수량,매입금액,매입단가,수익률,계좌
4,411060,ACE KRX금현물,1429220,15535,92,1330258,14459.33,7.43,64012548-22
1,360200,ACE 미국S&P500,1371800,18050,76,1338800,17615.79,2.46,64012548-22
7,456880,ACE 미국달러SOFR금리(합성),144690,11130,13,140018,10770.62,3.33,64012548-22
2,390390,KODEX 미국반도체MV,421470,20070,21,434120,20672.38,-2.91,64012548-22
9,483230,RISE 25-06 은행채(AA+이상)액티브,1862765,50345,37,1858510,50230.00,0.22,64012548-22
8,469050,RISE 미국반도체NYSE(H),461520,12820,36,477917,13275.47,-3.43,64012548-22
6,452360,SOL 미국배당다우존스(H),2055800,10820,190,1998428,10518.04,2.87,64012548-22
3,396500,TIGER Fn반도체TOP10,383940,10665,36,469560,13043.33,-18.23,64012548-22
5,449170,TIGER KOFR금리액티브(합성),3715950,106170,35,3703648,105818.51,0.33,64012548-22
0,329750,TIGER 미국달러단기채권액티브,1050175,12355,85,1013038,11918.09,3.66,64012548-22


In [101]:
total_2

12922245

In [102]:
df_3

,코드,이름,평가금액,단가,수량,매입금액,매입단가,수익률,계좌
2,411060,ACE KRX금현물,2004015,15535,129,1939575,15035.47,3.32,64043278-01
0,114800,KODEX 인버스,884000,4420,200,807893,4039.47,9.42,64043278-01
4,473330,SOL 미국30년국채커버드콜(합성),2191360,10240,214,2162650,10105.84,1.32,64043278-01
1,396500,TIGER Fn반도체TOP10,213300,10665,20,212050,10602.50,0.58,64043278-01
3,449170,TIGER KOFR금리액티브(합성),1911060,106170,18,1906737,105929.83,0.22,64043278-01
5,0,현금,257541,1,257541,257541,1.00,0.00,64043278-01


In [103]:
total_3

7461276

In [104]:
df_4

,코드,이름,평가금액,단가,수량,매입금액,매입단가,수익률,계좌
4,411060,ACE KRX금현물,4940130,15535,318,4770868,15002.73,3.55,64012548-29
3,365780,ACE 국고채10년,274500,91500,3,261365,87121.67,5.03,64012548-29
2,360200,ACE 미국S&P500,1732800,18050,96,1660065,17292.34,4.38,64012548-29
6,456880,ACE 미국달러SOFR금리(합성),1669500,11130,150,1660550,11070.33,0.54,64012548-29
1,284430,KODEX 200미국채혼합,518200,12955,40,504351,12608.77,2.75,64012548-29
8,468380,KODEX iShares미국하이일드액티브,1417000,10900,130,1418100,10908.46,-0.08,64012548-29
0,283580,KODEX 차이나CSI300,208600,10430,20,202350,10117.50,3.09,64012548-29
7,464540,RISE 25-03 회사채(AA-이상)액티브,8517835,51005,167,8507035,50940.33,0.13,64012548-29
9,473330,SOL 미국30년국채커버드콜(합성),3072000,10240,300,2984859,9949.53,2.92,64012548-29
5,449170,TIGER KOFR금리액티브(합성),10617000,106170,100,10573036,105730.36,0.42,64012548-29


In [105]:
total_4

34178071

In [106]:
# 그룹화하여 조건에 맞게 계산
result = df.groupby('코드').agg({
    '이름': 'first',
    '평가금액': 'sum',
    '단가': 'mean',
    '수량': 'sum',
    '매입금액': 'sum',
    '매입단가': 'mean',
}).reset_index()

result["비중"] = result["평가금액"] / result["평가금액"].sum() * 100
result["수익률"] = (result["평가금액"] - result["매입금액"]) / result["매입금액"] * 100
result.dropna(inplace=True)

# 결과 출력
result

,코드,이름,평가금액,단가,수량,매입금액,매입단가,비중,수익률
0,0,현금,2396943,1.00,2396943,2396943,1.00,2.31,0.00
1,114800,KODEX 인버스,884000,4420.00,200,807893,4039.47,0.85,9.42
2,283580,KODEX 차이나CSI300,208600,10430.00,20,202350,10117.50,0.20,3.09
3,284430,KODEX 200미국채혼합,518200,12955.00,40,504351,12608.77,0.50,2.75
4,329750,TIGER 미국달러단기채권액티브,1050175,12355.00,85,1013038,11918.09,1.01,3.67
5,360200,ACE 미국S&P500,8573750,18050.00,475,8329545,17500.38,8.27,2.93
6,365780,ACE 국고채10년,274500,91500.00,3,261365,87121.67,0.26,5.03
7,390390,KODEX 미국반도체MV,421470,20070.00,21,434120,20672.38,0.41,-2.91
8,396500,TIGER Fn반도체TOP10,2655585,10665.00,249,3215405,12258.10,2.56,-17.41
9,411060,ACE KRX금현물,15364115,15535.00,989,14710152,14829.63,14.82,4.45


In [107]:
eval_total = result['평가금액'].sum()
eval_total

103680388

In [108]:
# 카테고리 분류
# 114800	KODEX 인버스
# 283580	KODEX 차이나CSI300
# 284430	KODEX 200미국채혼합
# 329750	TIGER 미국달러단기채권액티브

원화현금 = ["0", "449170",]
달러현금 = ["456880",]
국내주식 = ["396500", "114800"]
미국주식 = ["360200", "390390", "402970", "469050", "480020", "482730", "446720", "452360"]
해외주식 = ["283580", ]
미국채권 = ["284430", "329750", "473330", "476750"]
국내채권 = ["365780", "464540", "483230"]
하이일드 = ["468380", ]
대채투자 = ["411060", ]

result['카테고리'] = '기타'
result.loc[result['코드'].isin(원화현금), '카테고리'] = '원화현금'
result.loc[result['코드'].isin(달러현금), '카테고리'] = '달러현금'
result.loc[result['코드'].isin(국내주식), '카테고리'] = '국내주식'
result.loc[result['코드'].isin(미국주식), '카테고리'] = '미국주식'
result.loc[result['코드'].isin(해외주식), '카테고리'] = '해외주식'
result.loc[result['코드'].isin(미국채권), '카테고리'] = '미국채권'
result.loc[result['코드'].isin(국내채권), '카테고리'] = '국내채권'
result.loc[result['코드'].isin(하이일드), '카테고리'] = '하이일드'
result.loc[result['코드'].isin(대채투자), '카테고리'] = '대채투자'
result.sort_values(by='카테고리', inplace=True)
result

,코드,이름,평가금액,단가,수량,매입금액,매입단가,비중,수익률,카테고리
1,114800,KODEX 인버스,884000,4420.00,200,807893,4039.47,0.85,9.42,국내주식
8,396500,TIGER Fn반도체TOP10,2655585,10665.00,249,3215405,12258.10,2.56,-17.41,국내주식
21,483230,RISE 25-06 은행채(AA+이상)액티브,1862765,50345.00,37,1858510,50230.00,1.80,0.23,국내채권
14,464540,RISE 25-03 회사채(AA-이상)액티브,10915070,51005.00,214,10901530,50943.52,10.53,0.12,국내채권
6,365780,ACE 국고채10년,274500,91500.00,3,261365,87121.67,0.26,5.03,국내채권
13,456880,ACE 미국달러SOFR금리(합성),4819290,11130.00,433,4746299,10917.02,4.65,1.54,달러현금
9,411060,ACE KRX금현물,15364115,15535.00,989,14710152,14829.63,14.82,4.45,대채투자
19,480020,ACE 미국빅테크7+ 15%프리미엄분배(합성),2558920,10360.00,247,2901566,11747.23,2.47,-11.81,미국주식
16,469050,RISE 미국반도체NYSE(H),461520,12820.00,36,477917,13275.47,0.45,-3.43,미국주식
12,452360,SOL 미국배당다우존스(H),4219800,10820.00,390,4136512,10604.23,4.07,2.01,미국주식


In [109]:
# 새로운 '분류' 컬럼 추가
result['대분류'] = result['카테고리']
result.loc[result['카테고리'].str.contains('현금'), '대분류'] = '현금'
result.loc[result['카테고리'].str.contains('주식'), '대분류'] = '주식'
result.loc[result['카테고리'].str.contains('채권'), '대분류'] = '채권'
result.loc[result['카테고리'].str.contains('하이일드'), '대분류'] = '채권'
result

,코드,이름,평가금액,단가,수량,매입금액,매입단가,비중,수익률,카테고리,대분류
1,114800,KODEX 인버스,884000,4420.00,200,807893,4039.47,0.85,9.42,국내주식,주식
8,396500,TIGER Fn반도체TOP10,2655585,10665.00,249,3215405,12258.10,2.56,-17.41,국내주식,주식
21,483230,RISE 25-06 은행채(AA+이상)액티브,1862765,50345.00,37,1858510,50230.00,1.80,0.23,국내채권,채권
14,464540,RISE 25-03 회사채(AA-이상)액티브,10915070,51005.00,214,10901530,50943.52,10.53,0.12,국내채권,채권
6,365780,ACE 국고채10년,274500,91500.00,3,261365,87121.67,0.26,5.03,국내채권,채권
13,456880,ACE 미국달러SOFR금리(합성),4819290,11130.00,433,4746299,10917.02,4.65,1.54,달러현금,현금
9,411060,ACE KRX금현물,15364115,15535.00,989,14710152,14829.63,14.82,4.45,대채투자,대채투자
19,480020,ACE 미국빅테크7+ 15%프리미엄분배(합성),2558920,10360.00,247,2901566,11747.23,2.47,-11.81,미국주식,주식
16,469050,RISE 미국반도체NYSE(H),461520,12820.00,36,477917,13275.47,0.45,-3.43,미국주식,주식
12,452360,SOL 미국배당다우존스(H),4219800,10820.00,390,4136512,10604.23,4.07,2.01,미국주식,주식


In [110]:
# 카테고리 별 비중 계산
대분류 = result.groupby('대분류').agg({
    '평가금액': 'sum',
    '단가': 'mean',
    '수량': 'sum',
    '매입금액': 'sum',
}).reset_index()

total_amount = 대분류['평가금액'].sum()
대분류['비중'] = 대분류['평가금액'] / total_amount * 100
대분류["수익률"] = (대분류["평가금액"] - 대분류["매입금액"]) / 대분류["매입금액"] * 100
대분류

,대분류,평가금액,단가,수량,매입금액,비중,수익률
0,대채투자,15364115,15535.00,989,14710152,14.82,4.45
1,주식,30636720,11802.50,2703,31563007,29.55,-2.93
2,채권,34219310,31203.75,2274,33335116,33.00,2.65
3,현금,23460243,39100.33,2397529,23326663,22.63,0.57


In [111]:
# 카테고리 별 비중 계산
category_group = result.groupby('카테고리').agg({
    '평가금액': 'sum',
    '단가': 'mean',
    '수량': 'sum',
    '매입금액': 'sum',
}).reset_index()

total_amount = category_group['평가금액'].sum()
category_group['비중'] = category_group['평가금액'] / total_amount * 100
category_group["수익률"] = (category_group["평가금액"] - category_group["매입금액"]) / category_group["매입금액"] * 100
category_group

,카테고리,평가금액,단가,수량,매입금액,비중,수익률
0,국내주식,3539585,7542.50,449,4023298,3.41,-12.02
1,국내채권,13052335,64283.33,254,13021405,12.59,0.24
2,달러현금,4819290,11130.00,433,4746299,4.65,1.54
3,대채투자,15364115,15535.00,989,14710152,14.82,4.45
4,미국주식,26888535,13215.71,2234,27337359,25.93,-1.64
5,미국채권,18114975,11470.00,1740,17260561,17.47,4.95
6,원화현금,18640953,53085.50,2397096,18580364,17.98,0.33
7,하이일드,3052000,10900.00,280,3053150,2.94,-0.04
8,해외주식,208600,10430.00,20,202350,0.20,3.09


In [112]:
# '대분류'가 '채권'인 항목 필터링
bond_df = result[result['대분류'] == '채권']

# 총 평가금액 계산
total_bond_amount = bond_df['평가금액'].sum()

# 각각의 항목에 대해 비중 계산
bond_df.loc[:, '비중'] = bond_df['평가금액'] / total_bond_amount * 100
bond_df

,코드,이름,평가금액,단가,수량,매입금액,매입단가,비중,수익률,카테고리,대분류
21,483230,RISE 25-06 은행채(AA+이상)액티브,1862765,50345.00,37,1858510,50230.00,5.44,0.23,국내채권,채권
14,464540,RISE 25-03 회사채(AA-이상)액티브,10915070,51005.00,214,10901530,50943.52,31.90,0.12,국내채권,채권
6,365780,ACE 국고채10년,274500,91500.00,3,261365,87121.67,0.80,5.03,국내채권,채권
4,329750,TIGER 미국달러단기채권액티브,1050175,12355.00,85,1013038,11918.09,3.07,3.67,미국채권,채권
3,284430,KODEX 200미국채혼합,518200,12955.00,40,504351,12608.77,1.51,2.75,미국채권,채권
17,473330,SOL 미국30년국채커버드콜(합성),15513600,10240.00,1515,14841509,9913.23,45.34,4.53,미국채권,채권
18,476750,ACE 미국30년국채엔화노출액티브(H),1033000,10330.00,100,901663,9016.63,3.02,14.57,미국채권,채권
15,468380,KODEX iShares미국하이일드액티브,3052000,10900.00,280,3053150,10904.40,8.92,-0.04,하이일드,채권


In [113]:
# '대분류'가 '주식'인 항목 필터링
stock_df = result[result['대분류'] == '주식']

# 총 평가금액 계산
total_stock_amount = stock_df['평가금액'].sum()

# 각각의 항목에 대해 비중 계산
stock_df.loc[:, '비중'] = stock_df['평가금액'] / total_stock_amount * 100
stock_df

,코드,이름,평가금액,단가,수량,매입금액,매입단가,비중,수익률,카테고리,대분류
1,114800,KODEX 인버스,884000,4420.00,200,807893,4039.47,2.89,9.42,국내주식,주식
8,396500,TIGER Fn반도체TOP10,2655585,10665.00,249,3215405,12258.10,8.67,-17.41,국내주식,주식
19,480020,ACE 미국빅테크7+ 15%프리미엄분배(합성),2558920,10360.00,247,2901566,11747.23,8.35,-11.81,미국주식,주식
16,469050,RISE 미국반도체NYSE(H),461520,12820.00,36,477917,13275.47,1.51,-3.43,미국주식,주식
12,452360,SOL 미국배당다우존스(H),4219800,10820.00,390,4136512,10604.23,13.77,2.01,미국주식,주식
10,446720,SOL 미국배당다우존스,3190500,10635.00,300,3136694,10455.65,10.41,1.72,미국주식,주식
7,390390,KODEX 미국반도체MV,421470,20070.00,21,434120,20672.38,1.38,-2.91,미국주식,주식
5,360200,ACE 미국S&P500,8573750,18050.00,475,8329545,17500.38,27.99,2.93,미국주식,주식
20,482730,TIGER 미국S&P500+10%프리미엄초단기,7462575,9755.00,765,7921005,10354.25,24.36,-5.79,미국주식,주식
2,283580,KODEX 차이나CSI300,208600,10430.00,20,202350,10117.50,0.68,3.09,해외주식,주식


In [114]:
# '대분류'가 '현금'인 항목 필터링
cash_df = result[result['대분류'] == '현금']

# 총 평가금액 계산
total_cash_amount = cash_df['평가금액'].sum()

# 각각의 항목에 대해 비중 계산
cash_df.loc[:, '비중'] = cash_df['평가금액'] / total_cash_amount * 100
cash_df

,코드,이름,평가금액,단가,수량,매입금액,매입단가,비중,수익률,카테고리,대분류
13,456880,ACE 미국달러SOFR금리(합성),4819290,11130.00,433,4746299,10917.02,20.54,1.54,달러현금,현금
11,449170,TIGER KOFR금리액티브(합성),16244010,106170.00,153,16183421,79369.68,69.24,0.37,원화현금,현금
0,0,현금,2396943,1.00,2396943,2396943,1.00,10.22,0.00,원화현금,현금



## 최근 영업일

In [115]:
import requests as rq
from bs4 import BeautifulSoup

url = 'https://finance.naver.com/sise/sise_deposit.nhn'
data = rq.get(url)
data_html = BeautifulSoup(data.content)
parse_day = data_html.select_one(
    'div.subtop_sise_graph2 > ul.subtop_chart_note > li > span.tah').text

print(parse_day)

  |  2024.08.06


In [116]:
import re

biz_day = re.findall('[0-9]+', parse_day)
biz_day = ''.join(biz_day)

print(biz_day)

20240806
